In [2]:
# part 1
import re
lines = open('inputs/input_16_test.txt','r').read().splitlines()
# lines = open('inputs/input_16.txt','r').read().splitlines()

valves = {}
for line in lines:
    doubles = [x for x in re.findall(r'([A-Z]{2})', line)]
    a = re.search(r'\d+', line) # find integer 
    flow = int(a.group(0))
    valves[doubles[0]] = flow,doubles[1:]

[print(x,valves[x]) for x in valves]


AA (0, ['DD', 'II', 'BB'])
BB (13, ['CC', 'AA'])
CC (2, ['DD', 'BB'])
DD (20, ['CC', 'AA', 'EE'])
EE (3, ['FF', 'DD'])
FF (0, ['EE', 'GG'])
GG (0, ['FF', 'HH'])
HH (22, ['GG'])
II (0, ['AA', 'JJ'])
JJ (21, ['II'])


[None, None, None, None, None, None, None, None, None, None]

In [13]:
from heapq import heapify, heappush, heappop

def move(dP, clock, loc, openset):
    for newloc in valves[loc][1]:
        if (loc,openset) not in visited:
            visited[(loc,openset)] = clock
            heappush(todo,(dP, clock, newloc, openset))
        elif clock < visited[(loc,openset)]:
            visited[(loc,openset)] = clock
            heappush(todo,(dP, clock, newloc, openset))
                
def openvalve(dP, clock, loc, openset):
    if loc not in openset:
        dP -= valves[loc][0]*(30-clock)
        heappush(todo, (dP, clock, loc, (*openset,loc)))

print(valves)
# completed = [] # heap of completed states
dP_biggest_drop = 0
visited = {}  # dictionary of cache of visited, key: loc,openset, value: clock
todo = [(0,0,'AA',())]  # starting state
clock = 0
while todo != []:
    dP, clock, loc, openset = heappop(todo)
    if dP < dP_biggest_drop:
        dP_biggest_drop = dP
        print(f'Best so far: {dP, clock, loc, openset}')
    clock += 1
    if clock < 5:
        openvalve(dP, clock, loc, openset)
        move(dP, clock, loc, openset)
        print(todo)
        print()
        print(visited)



{'AA': (0, ['DD', 'II', 'BB']), 'BB': (13, ['CC', 'AA']), 'CC': (2, ['DD', 'BB']), 'DD': (20, ['CC', 'AA', 'EE']), 'EE': (3, ['FF', 'DD']), 'FF': (0, ['EE', 'GG']), 'GG': (0, ['FF', 'HH']), 'HH': (22, ['GG']), 'II': (0, ['AA', 'JJ']), 'JJ': (21, ['II'])}
[(0, 1, 'AA', ('AA',)), (0, 1, 'DD', ())]

{('AA', ()): 1}


KeyboardInterrupt: 

In [9]:
print(clock)
print(todo)

6
[]
